In [1]:
#huggingface
from datasets import load_dataset
#torch data
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
#huggingface 
from transformers import BertTokenizer, BertModel
from transformers import AutoModel,AutoTokenizer
#torch nn
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import one_hot

#lightening
import pytorch_lightning as pl
from pytorch_lightning import Trainer

#metrics
from torchmetrics.functional import accuracy,recall,precision,f1_score





/root/miniconda3/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
#加载数据集Dataset
class MydataSet(Dataset):
    def __init__(self,path,split):
        self.dataset=load_dataset("csv",data_files=path,split=split)

    def __getitem__(self, index):
        text=self.dataset[index]["review"]
        label=self.dataset[index]["label"]
        return text,label
    def __len__(self):
        return len(self.dataset)
    


In [3]:
# data=MydataSet("./data/train.csv",'train')
# data[2]

In [4]:
#Dataloader生成迭代器
def collate_fn(data):
    
    # print(len(data)) #128
    sentence=[i[0] for i in data] #和参考代码不一样：[i[0] for i in data]
    label=[i[1] for i in data]

    data=token.batch_encode_plus(batch_text_or_text_pairs=sentence,
                                 max_length=200, #补全到长度200
                                 padding='max_length',
                                 pad_to_max_length=True,
                                 return_tensors='pt',
                                 truncation=True,
                                 return_length=True)
    input_ids=data['input_ids']
    attention_mask=data['attention_mask']
    token_type_ids=data['token_type_ids']
    label=torch.LongTensor(label)
    length=data['length']
    return input_ids,attention_mask,token_type_ids,label

In [5]:
#简单模型，测试各项数据和过程数据是否正常
class EasyModule(nn.Module):
    def __init__(self):
        super(EasyModule, self).__init__()
        self.drop=drop
        self.hidden_dim=hidden_dim
        self.output_dim=output_dim
        self.embedding=AutoModel.from_pretrained(bert_path)
        for param in self.embedding.parameters():
            param.requires_grad = False
        self.lstm=nn.LSTM(input_size=768,
                                    hidden_size=hidden_dim,
                                    num_layers=2,
                                    batch_first=True,
                                    bidirectional=True,
                                    dropout=self.drop)
        self.fc=nn.Linear(2*hidden_dim,self.output_dim)
    def forward(self, input_ids, attention_mask,token_type_ids):
        embedding=self.embedding(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        embedding=embedding.last_hidden_state
        
        print(embedding)


In [21]:
#实验用的模型
class BiLSTMmodule(nn.Module):
    def __init__(self,drop,hidden_dim,output_dim):
        super(BiLSTMmodule, self).__init__()
        self.drop=drop
        self.hidden_dim=hidden_dim
        self.output_dim=output_dim

        self.embedding=AutoModel.from_pretrained(local_files_only=True,pretrained_model_name_or_path=bert_path)#git lfs下载！！
        for param in self.embedding.parameters():
            param.requires_grad_(False)
        
        self.lstm=nn.LSTM(input_size=768,#312维的embedding??? 如果是bertbase是768维！
                            hidden_size=hidden_dim,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True,
                            dropout=self.drop)
        self.fc=nn.Linear(2*self.hidden_dim,self.output_dim) #BiLSTM，正向+反向
    def forward(self,input_ids,attention_mask,token_type_ids): #input_ids=attention_mask=[128,200] 
        embedded=self.embedding(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        embedded=embedded.last_hidden_state
        # print("embedded shape:"+str(embedded.shape)) # embedded shape:torch.Size([128, 200, 312])
        out,(h_n,c_n)=self.lstm(embedded)
        # print(out.shape,h_n.shape,c_n.shape)
        # out:([128, 200, 624]) h_n:([4, 128, 312]) c_n:([4, 128, 312])
        # print(out[:, -1, :312] == h_n[-2,:,:]) #True
        # print(out[:, 0, 312:] == h_n[-1, :, :]) #True
        # print(h_n[-1, :, :].shape) #torch.Size([128, 312])
        output=torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=1)#cat拼接函数，dim指示向哪个维度拼接
        # print("output shape"+str(output.shape))
        # output shape:torch.Size([128, 624])
        output=self.fc(output)
        # print(output.shape)
        return output
    


In [7]:
#lightening模型，管理整个训练过程
class BiLSTMLightening(pl.LightningModule):
    def __init__(self, drop,hidden_dim,output_dim):
        super(BiLSTMLightening,self).__init__()
        self.model = BiLSTMmodule(drop,hidden_dim,output_dim)
        self.criterion = nn.CrossEntropyLoss() #损失函数
        self.train_dataset=MydataSet("./data/train.csv",'train')
        self.val_dataset=MydataSet("./data/val.csv",'train')
        self.test_dataset=MydataSet("./data/test.csv",'train')
    def configure_optimizers(self):#优化器
        optimizer = optim.AdamW(self.parameters(), lr=lr)
        return optimizer
    def forward(self,input_ids,attention_mask,token_type_ids):
        return self.model(input_ids,attention_mask,token_type_ids)

    def train_datalader(self):
        train_loader=DataLoader(dataset=self.train_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)
        return train_loader

    def training_step(self,batch,batch_idx):
        input_ids,attention_mask,token_type_ids,label = batch #batch?
        y=one_hot(label,num_classes=class_num)
        y=y.to(dtype=torch.float)
        y_hat=self.model(input_ids,attention_mask,token_type_ids) #计算模型的输出
        y_hat=y_hat.squeeze()
        loss=self.criterion(y_hat,y) #对比计算损失
        self.log('traing_loss',loss,prog_bar=True,logger=True,on_step=True,on_epoch=True) #输出
        return loss
    
    def train_dataloader(self):
        train_loader=DataLoader(dataset=self.train_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn,num_workers=4)
        return train_loader
    
    def validation_step(self,batch,batch_idx):
        input_ids,attention_mask,token_type_ids,label = batch
        y=one_hot(label,num_classes=class_num)
        y=y.to(dtype=torch.float)
        y_hat=self.model(input_ids,attention_mask,token_type_ids)
        loss=self.criterion(y_hat,y)
        self.log('val_loss',loss,prog_bar=True,logger=True,on_step=True,on_epoch=True)
        return loss
    
    def val_dataloader(self):
        val_loader=DataLoader(dataset=self.val_dataset,batch_size=batch_size,shuffle=False,collate_fn=collate_fn,num_workers=4)
        return val_loader
    
    def test_dataloader(self):
        test_loader=DataLoader(dataset=self.test_dataset,batch_size=batch_size,shuffle=False,collate_fn=collate_fn,num_workers=4)
        return test_loader
    
    def test_step(self,batch,batch_idx):
        input_ids,attention_mask,token_type_ids,label = batch
        y=one_hot(label,num_classes=class_num)
        y=y.to(dtype=torch.float) #独热编码
        y_hat=self.model(input_ids,attention_mask,token_type_ids)
        y_hat=y_hat.squeeze() #测试的结果
        # print(y)
        # print(y_hat)
        # print(y_hat.shape)
        pred=torch.argmax(y_hat,dim=1) #返回最大行的下标
        acc=(pred==label).float().mean()
        loss=self.criterion(y_hat,y)
        # print(pred.shape)
        # print("label shape:"+str(label.shape))

        # task: Literal["binary", "multiclass", "multilabel"],对应[二分类，多分类，多标签]
        #  average=None分别输出各个类别, 不加默认算平均
        # re = recall(pred, label, task="", num_classes=class_num, average=None)
        # pre = precision(pred, label, task="binary", num_classes=class_num, average=None)
        # f1 = f1_score(pred, label, task="binary", num_classes=class_num, average=None)

        # def log_score(name, scores):
        #     for i, score_class in enumerate(scores):
        #         self.log(f"{name}_class{i}", score_class)

        # log_score("recall", re)
        # log_score("precision", pre)
        # log_score("f1", f1)
        self.log('test_loss', loss)
        self.log('acc',accuracy(preds=pred,target=label,task='binary')) #必须写task
        # self.log('acc', accuracy(pred, label, task="binary", num_classes=class_num))
        self.log('avg_recall',recall(preds=pred,target=label,task='binary',average='micro'))
        # # self.log('avg_recall', recall(pred, label, task="binary", num_classes=class_num, average="weighted"))
        self.log('avg_precision', precision(preds=pred, target=label,task='binary',average="weighted"))
        # self.log('avg_precision', precision(pred, label, task="binary", num_classes=class_num, average="weighted"))
        self.log('avg_f1', f1_score(pred, label, task="binary",average="weighted"))

In [13]:
def test():
    model=BiLSTMLightening.load_from_checkpoint(checkpoint_path=model_path,drop=dropout, hidden_dim=rnn_hidden, output_dim=class_num)
    trainer=Trainer(enable_checkpointing=False,fast_dev_run=False)
    trainer.test(model)

In [9]:
def train():
    trainer=Trainer(max_epochs=num_epochs,log_every_n_steps=10,accelerator='gpu',devices="auto")
    model=BiLSTMLightening(drop=dropout,hidden_dim=rnn_hidden,output_dim=class_num)
    trainer.fit(model=model)

In [ ]:

if __name__=="__main__":
    #超参数
    batch_size=256 #句子长度：200
    num_epochs=15
    dropout=0.5
    rnn_hidden=312
    rnn_layer=1
    class_num=2
    lr=0.001
    bert_path="/root/bert-base-chinese/"
    model_path="/root/train/lightning_logs/version_15(tiny_full)/checkpoints/epoch=14-step=4230.ckpt"

    # print(len(dataset))
    # print(dataset[0])
    # print(dataset[0][0])

    # dataset=MydataSet("./data/train.csv",'train')
    # loader=DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn,drop_last=True)
    
    # model=BiLSTMmodule(drop=dropout,hidden_dim=rnn_hidden,output_dim=class_num)
    # for i,(input_ids,attention_mask,token_type_ids,label) in enumerate(loader):
    #     print(attention_mask.shape)
    #     model.forward(input_ids,attention_mask,token_type_ids)
    #     break

    token=BertTokenizer.from_pretrained(bert_path)#用BertTokenizer
    train()



In [ ]:
#测试用
# if __name__=="__main__":
#     dataset=MydataSet("./data/train.csv",'train')
#     token=BertTokenizer.from_pretrained(bert_path)#用BertTokenizer
#     loader=DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn,drop_last=True)
#     model=BiLSTMmodule(drop=dropout,hidden_dim=rnn_hidden,output_dim=class_num)
#     for i,(input_ids,attention_mask,token_type_ids,label) in enumerate(loader):
#         print(attention_mask.shape)
#         model.forward(input_ids,attention_mask,token_type_ids)
#         break

        